In [18]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from matplotlib import pyplot as plt
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import ComplementNB
from sklearn.metrics import precision_score, recall_score, accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss
%matplotlib inline

In [19]:
data = pd.read_csv('../data/hr_analytics.csv')

In [20]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21287 entries, 0 to 21286
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   enrollee_id             21287 non-null  int64  
 1   city                    21287 non-null  object 
 2   city_development_index  21287 non-null  float64
 3   gender                  16271 non-null  object 
 4   relevent_experience     21287 non-null  object 
 5   enrolled_university     20870 non-null  object 
 6   education_level         20775 non-null  object 
 7   major_discipline        18162 non-null  object 
 8   experience              21217 non-null  object 
 9   company_size            14727 non-null  object 
 10  company_type            14513 non-null  object 
 11  last_new_job            20824 non-null  object 
 12  training_hours          21287 non-null  int64  
 13  target                  21287 non-null  float64
dtypes: float64(2), int64(2), object(10)
me

Как можно заметить датасет состоит в основном из категориальных признаков и только в них есть пропущенные значения, поэтому заменим их все на NA. Кроме того мы имеем бинарный признак "Пол", но обработаем его как обычный, так как пропусков в этом атрибуте много и попробуем модель обучить и для случая с неизвестным полом.

In [21]:
data = data.fillna('NA')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21287 entries, 0 to 21286
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   enrollee_id             21287 non-null  int64  
 1   city                    21287 non-null  object 
 2   city_development_index  21287 non-null  float64
 3   gender                  21287 non-null  object 
 4   relevent_experience     21287 non-null  object 
 5   enrolled_university     21287 non-null  object 
 6   education_level         21287 non-null  object 
 7   major_discipline        21287 non-null  object 
 8   experience              21287 non-null  object 
 9   company_size            21287 non-null  object 
 10  company_type            21287 non-null  object 
 11  last_new_job            21287 non-null  object 
 12  training_hours          21287 non-null  int64  
 13  target                  21287 non-null  float64
dtypes: float64(2), int64(2), object(10)
me

Заполнили пропуски и теперь перейдем к кодированию. Сначала создадим массив всех категориальных признаков для удобной работы и произведем кодирование.

In [29]:
cat = ['city', 'gender', 'relevent_experience', 'enrolled_university', 'education_level', 'major_discipline', 'experience', 'company_size', 'company_type', 'last_new_job']
data1 = pd.get_dummies(data[cat])
data_drop = data.drop(cat, axis=1)
data_final = pd.concat([data_drop, data1], axis=1)
data_final.to_csv('../data/hr_analytics_preprocessed.csv')
data_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21287 entries, 0 to 21286
Columns: 196 entries, enrollee_id to last_new_job_never
dtypes: float64(2), int64(2), uint8(192)
memory usage: 4.5 MB


In [23]:
data_final.head()

,enrollee_id,city_development_index,training_hours,target,city_city_1,city_city_10,city_city_100,city_city_101,city_city_102,city_city_103,...,company_type_Other,company_type_Public Sector,company_type_Pvt Ltd,last_new_job_1,last_new_job_2,last_new_job_3,last_new_job_4,last_new_job_>4,last_new_job_NA,last_new_job_never
0,8949,0.920,36,1.0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
1,29725,0.776,47,0.0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0
2,11561,0.624,83,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,33241,0.789,52,1.0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
4,666,0.767,8,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


Датасет перекодирован, теперь перейдем к разбиению на выборки и обучению моделей. Но сначала отделим признаки от целевого атрибута.

In [24]:
X = data_final.drop(['target'], axis=1)
y = data_final['target'].astype(int)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=0) #разобьем выборку в соотношении 7/3 и стратифицируем ее, кроме того установим random_state=0 для повторения данной операции

Начнем с логистической регрессии. Будем подбирать параметры с помощью крос-валидации. Найдем предсказания по тестовой выборке и оценим его по точности, полноте, доле правильных ответов, метриках f1, log_loss и auc. При сравнении будем основываться в первую очередь на метрике auc, так как остальные метрики оценивают модели с одной стороны, а log_loss в принципе сложно интерпретировать.

In [25]:
param_grid = {'C': [0.01, 0.05, 0.1, 0.5, 1, 5, 10]}
cv = 3
estimator = LogisticRegression(solver='liblinear')#Выбираем классификатор
optimizer = GridSearchCV(estimator, param_grid, cv=cv)#создаем сетку для поиска гиперпараметров
optimizer.fit(X_train, y_train)#обучаем классификатор
predict = optimizer.predict(X_test)#предсказываем результат по тестовой выборке

print('точность=', precision_score(y_test, predict))
print('полнота=', recall_score(y_test, predict))
print('доля правильных ответов=', accuracy_score(y_test, predict))
print('f1=', f1_score(y_test, predict))
print('log_loss=', log_loss(y_test, predict))
print('auc=', roc_auc_score(y_test, predict))

точность= 0.5772946859903382
полнота= 0.16678297278436846
количество правильных ответов= 0.7856583685611398
f1= 0.2587980508933405
log_loss= 7.40311958888101
auc= 0.5657289914386113


Получили качество примерно 56% посмотрим на другие модели.

In [26]:
param_grid = {'n_neighbors': [1, 3, 5, 10, 15],
              'leaf_size': [15, 30, 50, 100]}
cv = 3
estimator = KNeighborsClassifier()
optimizer = GridSearchCV(estimator, param_grid, cv=cv)
optimizer.fit(X_train, y_train)
predict = optimizer.predict(X_test)

print('точность=', precision_score(y_test, predict))
print('полнота=', recall_score(y_test, predict))
print('доля правильных ответов=', accuracy_score(y_test, predict))
print('f1=', f1_score(y_test, predict))
print('log_loss=', log_loss(y_test, predict))
print('auc=', roc_auc_score(y_test, predict))

точность= 0.25
полнота= 0.020237264480111653
количество правильных ответов= 0.7665570690465007
f1= 0.03744351194318915
log_loss= 8.06284408482672
auc= 0.5013378490345652


С помощью модели KNeighborsClassifier получили качество примерно 50%

In [27]:
param_grid = {'alpha': [0.1, 0.5, 1, 2, 5]}
cv = 3
estimator = ComplementNB()
optimizer = GridSearchCV(estimator, param_grid, cv=cv)
optimizer.fit(X_train, y_train)
predict = optimizer.predict(X_test)

print('точность=', precision_score(y_test, predict))
print('полнота=', recall_score(y_test, predict))
print('доля правильных ответов=', accuracy_score(y_test, predict))
print('f1=', f1_score(y_test, predict))
print('log_loss=', log_loss(y_test, predict))
print('auc=', roc_auc_score(y_test, predict))

точность= 0.2728122080348801
полнота= 0.6113049546406141
количество правильных ответов= 0.5472052606857679
f1= 0.3772609819121447
log_loss= 15.639268575869924
auc= 0.569984330368349


С помощью модели ComplementNB получили качество 57%

In [28]:
param_grid = {'C': [0.01, 0.05, 0.1, 0.5, 1, 5, 10]}
cv = 3
estimator = SVC()
optimizer = GridSearchCV(estimator, param_grid, cv=cv)
optimizer.fit(X_train, y_train)
predict = optimizer.predict(X_test)

print('точность=', precision_score(y_test, predict))
print('полнота=', recall_score(y_test, predict))
print('доля правильных ответов=', accuracy_score(y_test, predict))
print('f1=', f1_score(y_test, predict))
print('log_loss=', log_loss(y_test, predict))
print('auc=', roc_auc_score(y_test, predict))

точность= 0.0
полнота= 0.0
количество правильных ответов= 0.7756380147173947
f1= 0.0
log_loss= 7.749188441194146
auc= 0.5
C:\Users\evgen\AppData\Local\Programs\Python\Python38-32\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Получили качество 50%.
Теперь можем сделать вывод, что для наших данных и выбранных гиперпараметров лучшими моделями оказались LR и ComplementNB у них по метрике auc почество примерно 56-57%, что не является очень хорошим результатом для проверки хочет ли сменить работу человек или нет.